In [1]:
%env HF_ENDPOINT=https://hf-mirror.com
from KeepFIT.keepfit.modeling.model import KeepFITModel
from retriever import TextRetriever, MMRetriever
import torch
import os
'''
os.environ['http_proxy'] = 'http://127.0.0.1:1087'
os.environ['https_proxy'] = 'http://127.0.0.1:1087'
print("HTTP Proxy:", os.environ.get('http_proxy'))
print("HTTPS Proxy:", os.environ.get('https_proxy'))
'''

env: HF_ENDPOINT=https://hf-mirror.com


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'\nos.environ[\'http_proxy\'] = \'http://127.0.0.1:1087\'\nos.environ[\'https_proxy\'] = \'http://127.0.0.1:1087\'\nprint("HTTP Proxy:", os.environ.get(\'http_proxy\'))\nprint("HTTPS Proxy:", os.environ.get(\'https_proxy\'))\n'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mmRetriever=MMRetriever(device=device,dataset_path="/home/jqxu/Ragas/MM_feature_set")
textRetriever=TextRetriever(device=device,dataset_path="/home/jqxu/Ragas/text_datasets/dpr-txt/combined_books")
model = KeepFITModel(vision_type='resnet_v2', out_path='./output', from_checkpoint=True,
                        vision_pretrained=True,
                        weights_path=f'/home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth')
model.to(device)

Pretrained weights: None
Loaded pretrained weights from /mnt/data/public/MM_Retinal_Image/resnet_v2_epoch33_best.pth


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


100%|██████████| 6/6 [00:00<00:00, 442.48it/s]


Pretrained weights: None
Loaded pretrained weights from /mnt/data/public/MM_Retinal_Image/resnet_v2_epoch33_best.pth
load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


KeepFITModel(
  (vision_model): VisionModel(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): R

In [3]:
from extract.extractor import Extractor
from query_processor import QueryProcessor
import datasets
import numpy as np
import random
from PIL import Image
api_key = "REMOVED"
mmdataset = datasets.load_from_disk("/home/jqxu/Ragas/MM_feature_set")

keyWordExtractor=Extractor(model='gpt-3.5-turbo',api_key=api_key)
headers={
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
queryProcessor=QueryProcessor(headers)

In [4]:
import os
'''
random_indices=random.sample(range(len(mmdataset)), 20)
mmSamples = mmdataset.select(indices=random_indices)
'''
for sample in mmdataset:
    img_path = sample['image_path']
    img_name=os.path.basename(img_path)
    check_path = os.path.join("/home/jqxu/Ragas/datasets",img_name)
    if not os.path.exists(check_path):
        continue

    scores, retrieved_examples = mmRetriever.retrieve(img_path, k=3)
    similar_img_path = retrieved_examples["image_path"][1]
    similar_description = retrieved_examples["description"][0]
    #print("Image Description:",sample["description"])
    print("similar img_path",similar_img_path)
    
    #description=sample["description"]
    keywords=keyWordExtractor(similar_description)
    #print(f"extracted keywords: {keywords}")

    scores,retrieved_texts = textRetriever.retrieve(query=keywords, k=3,is_keywords=True)
    combined_texts = [f"{title}: {text}" for title, text in zip(retrieved_texts["title"],retrieved_texts["text"])]
    result=queryProcessor.process_query(image_path=img_path)
    """
    with open("keywords.txt", "a", encoding="utf-8") as f:
        f.write(f"Image path: {img_path}\n")
        f.write("Description:\n")
        f.write(description + "\n")
        f.write("Keywords:\n")
        f.write(keywords + "\n")
        f.write("Retrieved Texts:\n")
        for text in combined_texts:
            f.write(text + "\n")
            f.write("+" * 100 + "\n")
        f.write("-" * 100 + "\n")
    """
    #save the text to .txt file
    with open("./results_tmp/result_label.txt", "a", encoding="utf-8") as f:
        f.write(f"Image path: {img_path}\n")
        for choice in result['choices']:
            message_content = choice['message']['content']
            f.write(message_content + "\n")
            f.write("+" * 100 + "\n") 
            f.write("Combined Texts:\n")
            for text in combined_texts:
                f.write(text + "\n")
            f.write("-" * 100 + "\n") 
        print("-" * 100)

similar img_path /mnt/data/public/MM_Retinal_Image/MM_Retinal_dataset_v1/CFP/figure20-8-1A.png
['optic papillary edema', ' right eye']
['optic papillary edema', ' right eye']
Sending to OpenAI...
{'id': 'chatcmpl-AAt0hDdHpi0HgwWjavd0vSQoqhZcV', 'object': 'chat.completion', 'created': 1727158103, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "**Retinal Fundus Image Report**\n\n**Patient:** [Not Specified]\n**Date:** [Not Specified]\n\n**1. Optic Disc:**\nThe optic disc appears pale with blurred margins. This could be indicative of optic disc edema or papilledema. The irregular margins suggest swelling which might require further evaluation to rule out increased intracranial pressure or other pathologies.\n\n**2. Macula:**\nThe macula displays abnormal pigmentation with evident exudates around it. The central macula seems relatively less affected, but the surrounding deposits suggest macular edema or possibly diabetic maculopathy. The 

KeyboardInterrupt: 

In [5]:
import os
import shutil  # 用于复制相似图片

# 定义数据集路径
dataset_dir = "./datasets"  # 表示 datasets 目录在当前工作目录下
similar_dir = "./similarset"  # 用于存储相似图片的目录

# 确保 similarset 目录存在
if not os.path.exists(similar_dir):
    os.makedirs(similar_dir)

# 创建 HTML 文件
output_html = "all_images_report.html"
with open(output_html, "w", encoding="utf-8") as f:
    f.write("<html><body>\n")
    f.write("<h1>Retinal Images Report</h1>\n")

    # 图像路径过滤，获取所有图像文件的路径
    img_paths_filter = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(dataset_dir, filename)
            img_paths_filter.append(img_path)

    # 过滤出与 img_paths_filter 匹配的数据集样本
    filtered_dataset = mmdataset.filter(lambda x: os.path.basename(x['image_path']) in [os.path.basename(p) for p in img_paths_filter])

    # 遍历过滤后的数据集
    for sample in filtered_dataset:
        img_path = sample['image_path']  # 原始图像路径（相对路径或绝对路径）

        # 检索相似图像和描述
        try:
            scores, retrieved_examples = mmRetriever.retrieve(img_path, k=1)
            similar_img_path = retrieved_examples["image_path"][0]
            similar_description = retrieved_examples["description"][0]

            # 将相似图片复制到 similarset 目录
            similar_filename = os.path.basename(similar_img_path)  # 获取相似图片的文件名
            new_similar_path = os.path.join(similar_dir, similar_filename)
            if not os.path.exists(new_similar_path):
                shutil.copy(similar_img_path, new_similar_path)  # 复制相似图片到 similarset 目录

        except Exception as e:
            print(f"Error retrieving similar images for {img_path}: {e}")
            continue

        original_description = sample["description"]  # 从样本中获取原始描述

        # 将原始图像和相似图像及其描述写入 HTML
        f.write(f"<div style='margin-bottom: 40px;'>\n")

        # 写入图片名（文件名）并排
        f.write("<table style='width:100%;'>\n")
        f.write("<tr>\n")
        f.write(f"<td><strong>Original Image Name:</strong> {os.path.basename(img_path)}</td>\n")
        f.write(f"<td><strong>Similar Image Name:</strong> {similar_filename}</td>\n")
        f.write("</tr>\n")

        # 写入图片并排展示
        f.write("<tr>\n")
        f.write(f"<td><img src='{os.path.join(dataset_dir, os.path.basename(img_path))}' style='width:300px;'></td>\n")
        f.write(f"<td><img src='{os.path.join(similar_dir, similar_filename)}' style='width:300px;'></td>\n")
        f.write("</tr>\n")

        # 写入描述并排展示
        f.write("<tr>\n")
        f.write(f"<td><strong>Description:</strong> {original_description}</td>\n")
        f.write(f"<td><strong>Similar Description:</strong> {similar_description}</td>\n")
        f.write("</tr>\n")
        f.write("</table>\n")
        f.write("<hr>")  # 添加分隔线
        f.write("</div>\n")
    
    f.write("</body></html>\n")

print(f"HTML report saved as {output_html}")


HTML report saved as all_images_report.html
